Este agarra las imagenes de spectrogramas en spect/ y crea la matriz de grayscale (la estandariza, not forget!! see info.md) y la carga a un CSV (despues vemos donde)

In [2]:
import os, sys
import pandas as pd
import librosa as lbrs
import numpy as np
from PIL import Image

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import util

In [6]:
# Aux Functions
def load_TrainDataFrame(spects_source_dir, spects_meta_df): 
    ''' Takes the spectrograms from the spects_source_dir folder, and gets the images into a matrix form, 
    with the grayscale of all the pixels, so they can be later made tensors of compatible CNN training input. 
    It loads the matrices pixel info into a CSV so they can then be read & extracted, easily reconverted into 
    a feedable matrix without having to re-run this notebook again. The target CSV (at database/meta/train_data.csv) 
    should have a row for each sample, with the columns class_id, and then all the pixel elements of the spectrogram.

    Args:
        spects_source_dir (str): Directory where the spectrogram images are stored in .png format.
        spects_meta_csv (pd.DataFrame): DataFrame for each spectrogram, with columns 'filename' and 'class_id'.
        meta_csv_path (str): Path to the CSV file where the final data will be stored, with the matrix elements and the class_id labeling each sample
    '''
    data_rows = []
    spects_meta_df = spects_meta_df.dropna(subset=['filename', 'class_id'])

    print(f"Processing {len(spects_meta_df)} spectrograms...")
    processed_count = 0
    skipped_count = 0

    for _, row in spects_meta_df.iterrows():
        filename = row['filename']
        class_id = row['class_id']

        image_path = os.path.join(spects_source_dir, filename)
        if not os.path.exists(image_path):
            print(f"File not found: {image_path}")
            skipped_count += 1
            continue

        img = Image.open(image_path).convert('L')  # Ensure grayscale
        
        expected_shape = (313, 224)
        if img.size != expected_shape:
            raise ValueError(f"Unexpected image size: {img.size} in file {image_path}. Expected {expected_shape}.")

        pixels = np.array(img).flatten()
        data_row = [class_id] + pixels.tolist()
        data_rows.append(data_row)
        processed_count += 1

    print(f"Successfully processed: {processed_count}")
    print(f"Skipped: {skipped_count}")

    if not data_rows:
        raise ValueError("No spectrograms were loaded. Check paths and metadata consistency.")

    n_pixels = len(data_rows[0]) - 1
    columns = ['label'] + [f'px_{i}' for i in range(n_pixels)]
    df = pd.DataFrame(data_rows, columns=columns)

    return df

In [3]:
spect_dir = os.path.join('..', 'database', 'spect') # Has the spectrogram pngs
spects_df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final_spects.csv')) # Has the spectrogram metadata ('filename', 'class_id')
data_csv_path = os.path.join('..', 'database', 'meta', 'final', 'train_data.csv') # path where the final data_csv will be saved, with the grayscale pixel matrices and class ids

spect2_dir = os.path.join('..', 'database', 'spect2') # Has the spectrogram pngs for the second set of data
spects2_df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final_spects2.csv')) # Has the spectrogram metadata ('filename', 'class_id')
data_csv2_path = os.path.join('..', 'database', 'meta', 'final', 'train_data2.csv') # path where the final data_csv will be saved, with the grayscale pixel matrices and class ids

In [ ]:
# Table of how many samples per class
print(spects_df['class_id'].value_counts().sort_index().reset_index().rename(columns={'index': 'class_id', 'class_id': 'samples'}).to_string(index=False, header=['class', 'samples']))

class_id
0     101
1      73
2     213
3      90
4      38
5      76
6      40
7      95
8      52
9     113
10    144
11     42
12     17
13     27
14     70
15     45
16    360
17     99
18    143
19    132
20     62
21    111
22     76
23    168
24    280
25     37
26    150
27    216
Name: count, dtype: int64


Train Data 1 (not noise-reduced)

In [8]:
print("Loading DataFrame...")
train_df = load_TrainDataFrame(spect_dir, spects_df)

print("DataFrame loaded and saved successfully.")
print(f"DataFrame shape: {train_df.shape}")

Loading DataFrame...
Processing 3070 spectrograms...
Successfully processed: 3070
Skipped: 0
DataFrame loaded and saved successfully.
DataFrame shape: (3070, 70113)


In [9]:
print("Loading CSV...")
train_df.to_csv(data_csv_path, index=False)

Loading CSV...


Train Data 2 (noise-reduced)

In [11]:
print("Loading second DataFrame...")
train_df2 = load_TrainDataFrame(spect2_dir, spects2_df)

print("Second DataFrame loaded and saved successfully.")
print(f"Second DataFrame shape: {train_df2.shape}")

Loading second DataFrame...
Processing 3070 spectrograms...
Successfully processed: 3070
Skipped: 0
Second DataFrame loaded and saved successfully.
Second DataFrame shape: (3070, 70113)


In [12]:
print("Loading second CSV...")
train_df2.to_csv(data_csv2_path, index=False)

Loading second CSV...
